In [1]:
import os

# 设置文件夹路径和类别名称
folders = [r'C:\Users\bugs_\PycharmProjects\eegProject\data\Test_EEG\HC',
           r'C:\Users\bugs_\PycharmProjects\eegProject\data\Test_EEG\MDD',
           r'C:\Users\bugs_\PycharmProjects\eegProject\data\Test_EEG\BD']
class_names = ['HC', 'MDD', 'BD']

# 创建存储数据集的字典
data_file = {'filename': []}

# 遍历每个类别的文件夹
for folder, class_name in zip(folders, class_names):
    # 获取文件夹中的Clean.mat文件列表
    file_list = os.listdir(folder)
    file_list = [os.path.join(folder, file) for file in file_list if file.endswith('Clean.mat')]

    # 将数据加入到对应的数据集中
    data_file['filename'].extend([(file, class_name) for file in file_list])


In [2]:
import numpy as np
import scipy.io as sio

label_dic = {'HC': 0, 'MDD': 1, "BD": 2}
target_shape = (284, 16, 24000)
X = np.empty(target_shape)
y = {'label': np.empty(284, dtype=int)}

num_merged = 0

for sub, label in data_file['filename']:  # 被试循环

    data = sio.loadmat(sub)
    sample = data['EEG_ECClean']
    eeg_data = sample["data"][0][0][:, 0:24000]
    X[num_merged] = eeg_data
    y['label'][num_merged] = label_dic[label]
    num_merged += 1

In [3]:
from torcheeg import transforms
from torcheeg.datasets import NumpyDataset

dataset = NumpyDataset(X=X,
                       y=y,
                       io_path='./features/minmax',
                       online_transform=transforms.Compose([
                           transforms.MinMaxNormalize(axis=-1),
                           transforms.To2d(),
                           transforms.ToTensor()
                       ]),
                       label_transform=transforms.Select('label'),
                       num_worker=4
                       )
print(dataset[0]) 

The target folder already exists, if you need to regenerate the database IO, please delete the path ./features/minmax.
(tensor([[[0.4026, 0.4136, 0.4355,  ..., 0.3626, 0.3703, 0.3839],
         [0.3323, 0.3181, 0.3185,  ..., 0.3186, 0.3220, 0.3221],
         [0.4441, 0.4620, 0.4759,  ..., 0.4478, 0.4316, 0.4256],
         ...,
         [0.5149, 0.5226, 0.5340,  ..., 0.5443, 0.5509, 0.5471],
         [0.6058, 0.5844, 0.5736,  ..., 0.5885, 0.5782, 0.5836],
         [0.4875, 0.4923, 0.4864,  ..., 0.5046, 0.5055, 0.4986]]]), 0)


In [4]:
from torcheeg.model_selection import KFold

k_fold = KFold(n_splits=5,
               split_path=f'./tmp_out/split_5',
               shuffle=True)

In [5]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)
batch_size = 32

In [6]:
from torch.utils.tensorboard import writer

def train(dataloader, model, loss_fn, optimizer):
    global total_train_step
    size = len(dataloader.dataset)
    model.train()
    train_correct = 0

    for batch_idx, batch in enumerate(dataloader):
        X = batch[0].to(device)
        y = batch[1].to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1

        # if batch_idx % 100 == 0:
        loss, current = loss.item(), batch_idx * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        writer.add_scalar("train_loss", loss, total_train_step)
        train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_correct /= size
    writer.add_scalar("train_auc", train_correct, total_train_step)
    print(f"Train Error: \n Accuracy: {(100 * train_correct):>0.1f}% ")

def valid(dataloader, model, loss_fn):
    global total_val_step
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            X = batch[0].to(device)
            y = batch[1].to(device)

            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    correct /= size
    # writer.add_scalar("test_avg_loss", val_loss, total_val_step)
    writer.add_scalar("test_auc", correct, total_val_step)
    total_val_step += 1
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {val_loss:>8f} \n")

In [7]:
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.loader import DataLoader
from torcheeg.models import EEGNet

writer = SummaryWriter(r".\log\log_minmax_EEGNet64_16_8_16_2_drop5_5k_shuffle111_batch32_epoch500_lr1e-3")
total_train_step = 0
total_val_step = 0

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):

    model = EEGNet(chunk_size=24000,
               num_electrodes=16,
               dropout=0.5,
               kernel_1=64,
               kernel_2=16,
               F1=8,
               F2=16,
               D=2,
               num_classes=3).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    epochs = 500
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_loader, model, loss_fn, optimizer)
        valid(val_loader, model, loss_fn)
    print("Done!")
writer.close()

Epoch 1
-------------------------------
loss: 1.109416  [    0/  227]
loss: 1.662719  [   32/  227]
loss: 1.456535  [   64/  227]
loss: 1.204372  [   96/  227]
loss: 1.035339  [  128/  227]
loss: 1.177250  [  160/  227]
loss: 1.451735  [  192/  227]
loss: 1.308870  [   21/  227]
Train Error: 
 Accuracy: 44.1% 
Test Error: 
 Accuracy: 52.6%, Avg loss: 1.036450 

Epoch 2
-------------------------------
loss: 1.046732  [    0/  227]
loss: 0.877721  [   32/  227]
loss: 0.908434  [   64/  227]
loss: 0.898788  [   96/  227]
loss: 1.198524  [  128/  227]
loss: 0.953876  [  160/  227]
loss: 0.961595  [  192/  227]
loss: 0.872477  [   21/  227]
Train Error: 
 Accuracy: 54.2% 
Test Error: 
 Accuracy: 17.5%, Avg loss: 1.113818 

Epoch 3
-------------------------------
loss: 0.809502  [    0/  227]
loss: 0.876929  [   32/  227]
loss: 0.762437  [   64/  227]
loss: 0.858095  [   96/  227]
loss: 0.854137  [  128/  227]
loss: 0.830258  [  160/  227]
loss: 0.803105  [  192/  227]
loss: 0.680894  [   21